<a href="https://colab.research.google.com/github/skymined/study_AIs/blob/main/docs/quests/MLs/LetalCarOfContractType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quest
- age 결측치 처리 : linear regression
- feature는 수치형만 사용
- 공유 : 결측치를 채운 테이터셋, 소스, pickle

In [28]:
import pandas as pd
import numpy as np
df_RCOCT = pd.read_csv("/content/LetalCarOfContractType.csv")
df_RCOCT[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0


In [29]:
df_RCOCT['age'].isnull().sum()

10795

### features, target 선택
- target : age
- features : term, amount

In [30]:
df_RCOCT.isnull().sum()
# 결측치가 없는 term과 amount를 가지고 진행

id                       0
type_of_contract         4
type_of_contract2        1
channel                  0
datetime                 0
Term                     0
payment_type             0
product                  1
amount                   0
state                    0
overdue_count            0
overdue                  2
credit rating         8783
bank                  2760
cancellation            25
age                  10795
Mileage              10795
dtype: int64

In [33]:
# age에 있는 null 값들은 우선 지워야함
df_RCOCT_Regression = df_RCOCT.dropna(subset=['age'])
df_RCOCT_Regression.isnull().sum()

id                    0
type_of_contract      3
type_of_contract2     1
channel               0
datetime              0
Term                  0
payment_type          0
product               1
amount                0
state                 0
overdue_count         0
overdue               2
credit rating         0
bank                  2
cancellation         20
age                   0
Mileage               0
dtype: int64

### 데이터 분리
- target과 features 분리

In [38]:
target = df_RCOCT_Regression['age']
features = df_RCOCT_Regression[['Term', 'amount']]
target.shape, features.shape

((40509,), (40509, 2))

In [39]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=2)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((28356, 2), (12153, 2), (28356,), (12153,))

### 모델 뚝딱뚝딱

In [40]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [41]:
# 학습 시키기
model.fit(features_train, target_train)

LinearRegression()

In [42]:
from sklearn.metrics import r2_score
target_train_predict = model.predict(features_train)
target_train.shape, target_train_predict.shape

((28356,), (28356,))

In [43]:
r2_score(target_train, target_train_predict)

1.0724032952325047e-05

In [44]:
target_test_predict =model.predict(features_test)
target_test.shape, target_test_predict.shape

((12153,), (12153,))

In [45]:
r2_score(target_test, target_test_predict)

9.945969531022225e-05

### 데이터셋 넣어보기

In [60]:
features_real = df_RCOCT.query('age.isna()')
predicted_age = model.predict(features_real[['Term', 'amount']])

In [62]:
df_RCOCT.loc[df_RCOCT['age'].isna(), 'age'] =predicted_age

In [63]:
df_RCOCT.isnull().sum()

id                       0
type_of_contract         4
type_of_contract2        1
channel                  0
datetime                 0
Term                     0
payment_type             0
product                  1
amount                   0
state                    0
overdue_count            0
overdue                  2
credit rating         8783
bank                  2760
cancellation            25
age                      0
Mileage              10795
dtype: int64

### 피클 만들기

In [46]:
import pickle
with open('RentalCarOfContractType.pkl', 'wb') as pickle_file :
  pickle.dump(obj=model, file=pickle_file)

In [65]:
df_RCOCT.to_csv('RentalCarOfContractType_ageRegression.csv', index=False)